In [44]:
import pickle
import pandas as pd
import numpy as np

一個禮拜中某isp各protocols的session總數

In [46]:
isp = '台灣固網'
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]  #改!!
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))

In [47]:
noise_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/noise_cluster.pkl'
denoise_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/denoise_cluster.pkl'
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'
all_df = pickle.load(open(pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl','rb'))
noise_clusters = pickle.load(open(noise_path,'rb'))
denoise_clusters = pickle.load(open(denoise_path,'rb'))
assert len(noise_clusters) + len(denoise_clusters) == len(all_df)
all_df = all_df[~all_df.idx.isin(noise_clusters)]
# all_df = all_df[all_df.idx.isin(denoise_clusters)]
# all_df = all_df.reset_index(drop=True)
all_df

,idx,timestamp,country,src_ip,ssdeep
0,20200106_15733_http,"[1578239430.546177, 1578239430.627715, 1578239...","[United States, United States, United States, ...","[66.249.69.175, 66.249.69.145, 66.249.71.5, 66...",[48:+eWa5lO918hw18VcM18xdnXtctdLJCAY6S:+eXg91u...
2,20200106_177_http,"[1578239428.145726, 1578239432.99058, 15782394...","[Netherlands, United States, Iran, United Stat...","[169.197.108.38, 216.244.66.198, 217.60.228.17...",[12:rHXq6xKv89IshSR56Y1awh5UPlVFYNjZpvhm:Da5kW...
3,20200106_51539642503_http,"[1578248664.635621, 1578248842.672464, 1578249...","[Hong Kong, Egypt, Hong Kong, China, China, Ch...","[103.90.203.174, 193.227.5.230, 103.90.203.174...",[12:7KS3TXcHnMQ7s95HfUzUfszUfsYjmHXq/Ish+j51h1...
6,20200106_136_http,"[1578239430.724066, 1578239431.97968, 15782394...","[China, China, China, Taiwan, China, China, Ch...","[106.13.108.181, 59.38.126.238, 59.38.126.238,...",[12:7QoEmHXq/I75cuXNFljhgDxKv8rDH46ZY1Zu8m:7UA...
7,20200106_2386_http,"[1578241539.855242, 1578241926.134562, 1578241...","[United States, France, France, France, France...","[108.52.243.101, 147.135.130.142, 147.135.130....",[12:7iotDpiNOmHXqzNkZr1NkWQwHj1NkZfsvq7ALTbxKv...
...,...,...,...,...,...
630,20200110_42949697374_telnet,"[1578641181.71725, 1578664950.870665]","[United States, United States]","[204.93.154.208, 204.93.154.194]","[3:ro53fg5T:ro2N, 3:ro53fg5T:ro2N, 3:ro53fg5T:..."
631,20200108_94489305705_sip,"[1578455241.326382, 1578455241.327904, 1578455...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:p2kQ17mDETU7ft9ckQFAOckQFA+UDR0dSRtPKiRvb3...
632,20200106_10024_sip,"[1578293821.543121, 1578293821.543636, 1578293...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[48:Yoaj0rXuXugIAO08oKY18Vje821BCkOPk/Ix:YnoXu...
633,20200106_1876_sip,"[1578239432.53756, 1578239442.108728, 15782394...","[United States, United States, United States, ...","[104.206.128.22, 104.206.128.22, 104.140.188.2...",[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...


In [48]:
def cut_protocol(cluster_id):
    '''
    GOAL: 將cluster index轉換為protocol名稱
    '''
    return cluster_id.split("_")[-1]
def count_sessions_num(time_li):
    '''
    GOAL: 依據timestamp list計算每個cluster所具有的session數量
    '''
    return len(time_li)
all_df['protocol'] = all_df.idx.map(cut_protocol)
all_df['session_num'] = all_df.timestamp.map(count_sessions_num)
all_df

,idx,timestamp,country,src_ip,ssdeep,protocol,session_num
0,20200106_15733_http,"[1578239430.546177, 1578239430.627715, 1578239...","[United States, United States, United States, ...","[66.249.69.175, 66.249.69.145, 66.249.71.5, 66...",[48:+eWa5lO918hw18VcM18xdnXtctdLJCAY6S:+eXg91u...,http,331276
2,20200106_177_http,"[1578239428.145726, 1578239432.99058, 15782394...","[Netherlands, United States, Iran, United Stat...","[169.197.108.38, 216.244.66.198, 217.60.228.17...",[12:rHXq6xKv89IshSR56Y1awh5UPlVFYNjZpvhm:Da5kW...,http,206866
3,20200106_51539642503_http,"[1578248664.635621, 1578248842.672464, 1578249...","[Hong Kong, Egypt, Hong Kong, China, China, Ch...","[103.90.203.174, 193.227.5.230, 103.90.203.174...",[12:7KS3TXcHnMQ7s95HfUzUfszUfsYjmHXq/Ish+j51h1...,http,193379
6,20200106_136_http,"[1578239430.724066, 1578239431.97968, 15782394...","[China, China, China, Taiwan, China, China, Ch...","[106.13.108.181, 59.38.126.238, 59.38.126.238,...",[12:7QoEmHXq/I75cuXNFljhgDxKv8rDH46ZY1Zu8m:7UA...,http,46402
7,20200106_2386_http,"[1578241539.855242, 1578241926.134562, 1578241...","[United States, France, France, France, France...","[108.52.243.101, 147.135.130.142, 147.135.130....",[12:7iotDpiNOmHXqzNkZr1NkWQwHj1NkZfsvq7ALTbxKv...,http,33540
...,...,...,...,...,...,...,...
630,20200110_42949697374_telnet,"[1578641181.71725, 1578664950.870665]","[United States, United States]","[204.93.154.208, 204.93.154.194]","[3:ro53fg5T:ro2N, 3:ro53fg5T:ro2N, 3:ro53fg5T:...",telnet,2
631,20200108_94489305705_sip,"[1578455241.326382, 1578455241.327904, 1578455...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:p2kQ17mDETU7ft9ckQFAOckQFA+UDR0dSRtPKiRvb3...,sip,3901
632,20200106_10024_sip,"[1578293821.543121, 1578293821.543636, 1578293...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[48:Yoaj0rXuXugIAO08oKY18Vje821BCkOPk/Ix:YnoXu...,sip,2277
633,20200106_1876_sip,"[1578239432.53756, 1578239442.108728, 15782394...","[United States, United States, United States, ...","[104.206.128.22, 104.206.128.22, 104.140.188.2...",[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,sip,1917


In [49]:
print(isp,"經濾除noiseClusters後在",min_date,"至",max_date,"的期間中各protocols的session數量:")
#all_df.groupby("protocol")['timestamp'].sum().apply(len) #alternative
temp = pd.DataFrame(all_df.groupby('protocol')['session_num'].sum()).sort_values('session_num',ascending=False)
temp#.reset_index()

台灣固網 經濾除noiseClusters後在 20200106 至 20200112 的期間中各protocols的session數量:


,session_num
protocol,
ssh,9432064
smb,1864386
http,993223
smtp,131451
ftp,44616
mysql,26498
pop,9295
rpc,9150
sip,8153
